# Sympy decomposotition code

In [19]:
import sympy as sp

In [20]:
# Define symbolic Pauli matrices
I = sp.Matrix([[1, 0], [0, 1]])
X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])
pauli_matrices = [I, X, Y, Z]
pauli_labels = ["I", "X", "Y", "Z"]

In [21]:
# Function to compute coefficients for symbolic decomposition
def decompose_4x4_symbolic(M):
    coefficients = {}
    for i, P1 in enumerate(pauli_matrices):
        for j, P2 in enumerate(pauli_matrices):
            basis_matrix = sp.kronecker_product(P1, P2)  # Tensor product
            coefficient = (1 / 4) * sp.trace(M * basis_matrix)  # Compute symbolic coefficient
            coefficients[f"{pauli_labels[i]}{pauli_labels[j]}"] = sp.simplify(coefficient)
    # Remove zero coefficients
    coefficients = {key: value for key, value in coefficients.items() if value != 0}
    return coefficients

In [22]:
def decompose_4x4_symbolic_new(M):
    coefficients = []
    for i, P1 in enumerate(pauli_matrices):
        for j, P2 in enumerate(pauli_matrices):
            basis_matrix = sp.kronecker_product(P1, P2)  # Tensor product
            coefficient = (1 / 4) * sp.trace(M * basis_matrix)  # Compute symbolic coefficient
            simplified_coefficient = sp.simplify(coefficient)
            if simplified_coefficient != 0:
                coefficients.append((simplified_coefficient, f"{pauli_labels[i]}{pauli_labels[j]}"))
    return coefficients

In [23]:
# Function to compute the commutator
def commutator(A, B):
    return A * B - B * A

# Function to compute the anticommutator
def anticommutator(A, B):
    return A * B + B * A

def lindblad_term(O, L):
    """Dissipative Lindblad term."""
    term1 = sp.conjugate(sp.transpose(L)) * O * L # L_k^\dagger * O * L
    term2 = anticommutator(O, sp.conjugate(sp.transpose(L)) * L) / 2  # {O, L_k^\dagger L_k} / 2
    return term1 - term2

In [24]:
# Main EDP function
def EDP(H, O, Ls, gammas):
    """
    Compute the time derivative of <O> and decompose it into Pauli tensor products.

    H: Hamiltonian (symbolic matrix)
    O: Observable (symbolic matrix)
    Ls: List of Lindblad operators (symbolic matrices)
    gammas: List of decay rates associated with Lindblad operators (symbolic values)

    Returns:
        Dictionary of coefficients for the Pauli tensor product decomposition.
    """
    # Unitary part: -i * [H, O]
    unitary_part = (-1) * 1j * commutator(O, H)

    # Dissipative part
    dissipative_part = sp.zeros(4)
    for gamma_k, L_k in zip(gammas, Ls):
        L = gamma_k * lindblad_term(O, L_k)
        dissipative_part += L

    # Total time derivative
    dO_dt = unitary_part + dissipative_part

    # Decompose into Pauli tensor products
    decomposition = decompose_4x4_symbolic_new(dO_dt)

    return decomposition

In [25]:
# Define the Lindblad dissipation
gamma1, gamma2, gamma3, gamma4 = sp.symbols("gamma1 gamma2 gamma3, gamma4", real=True, positive=True)
gammas = [gamma1, gamma2, gamma3, gamma4]

L1 = sp.kronecker_product((X - 1j*Y)/2, I)
L2 = sp.kronecker_product(Z, I)
L3 = sp.kronecker_product(I, (X - 1j*Y)/2)
L4 = sp.kronecker_product(I, Z)
Ls = [L1, L2, L3, L4]

In [26]:
# Define the Hamiltonian with external magnetic field
JXX,JXY,JXZ,JYX,JYY,JYZ,JZX,JZY,JZZ = sp.symbols('JXX,JXY,JXZ,JYX,JYY,JYZ,JZX,JZY,JZZ')
H=JXX * sp.kronecker_product(X, X)+JXY * sp.kronecker_product(X, Y)+JXZ * sp.kronecker_product(X, Z)
H+=JYX * sp.kronecker_product(Y, X)+JYY * sp.kronecker_product(Y, Y)+JYZ * sp.kronecker_product(Y, Z)
H+=JZX * sp.kronecker_product(Z, X)+JZY * sp.kronecker_product(Z, Y)+JZZ * sp.kronecker_product(Z, Z)
H=H/2
for i, P1 in enumerate(pauli_matrices):
    for j, P2 in enumerate(pauli_matrices):
        O = sp.kronecker_product(P1, P2)
        Eq = EDP(H, O, Ls, gammas)
        print(f'{pauli_labels[i]}{pauli_labels[j]} {Eq}')

II []
IX [(-0.5*gamma3 - 2.0*gamma4, 'IX'), (-1.0*JXZ, 'XY'), (JXY, 'XZ'), (-1.0*JYZ, 'YY'), (1.0*JYY, 'YZ'), (-1.0*JZZ, 'ZY'), (1.0*JZY, 'ZZ')]
IY [(-0.5*gamma3 - 2.0*gamma4, 'IY'), (1.0*JXZ, 'XX'), (-1.0*JXX, 'XZ'), (JYZ, 'YX'), (-1.0*JYX, 'YZ'), (1.0*JZZ, 'ZX'), (-JZX, 'ZZ')]
IZ [(-1.0*gamma3, 'II'), (-1.0*gamma3, 'IZ'), (-1.0*JXY, 'XX'), (1.0*JXX, 'XY'), (-1.0*JYY, 'YX'), (JYX, 'YY'), (-1.0*JZY, 'ZX'), (1.0*JZX, 'ZY')]
XI [(-0.5*gamma1 - 2.0*gamma2, 'XI'), (-1.0*JZX, 'YX'), (-1.0*JZY, 'YY'), (-1.0*JZZ, 'YZ'), (JYX, 'ZX'), (1.0*JYY, 'ZY'), (1.0*JYZ, 'ZZ')]
XX [(-1.0*JXZ, 'IY'), (1.0*JXY, 'IZ'), (-0.5*gamma1 - 2.0*gamma2 - 0.5*gamma3 - 2.0*gamma4, 'XX'), (-1.0*JZX, 'YI'), (1.0*JYX, 'ZI')]
XY [(JXZ, 'IX'), (-1.0*JXX, 'IZ'), (-0.5*gamma1 - 2.0*gamma2 - 0.5*gamma3 - 2.0*gamma4, 'XY'), (-1.0*JZY, 'YI'), (1.0*JYY, 'ZI')]
XZ [(-1.0*JXY, 'IX'), (1.0*JXX, 'IY'), (-1.0*gamma3, 'XI'), (-0.5*gamma1 - 2.0*gamma2 - 1.0*gamma3, 'XZ'), (-1.0*JZZ, 'YI'), (1.0*JYZ, 'ZI')]
YI [(1.0*JZX, 'XX'), (JZY, '

In [27]:
# Define the Hamiltonian with external magnetic field
JXX,JXY,JXZ,JYX,JYY,JYZ,JZX,JZY,JZZ = sp.symbols('JXX,JXY,JXZ,JYX,JYY,JYZ,JZX,JZY,JZZ')
H =JXX * sp.kronecker_product(X, X)+JXY * sp.kronecker_product(X, Y)+JXZ * sp.kronecker_product(X, Z)
H+=JYX * sp.kronecker_product(Y, X)+JYY * sp.kronecker_product(Y, Y)+JYZ * sp.kronecker_product(Y, Z)
H+=JZX * sp.kronecker_product(Z, X)+JZY * sp.kronecker_product(Z, Y)+JZZ * sp.kronecker_product(Z, Z)
H=H/2
countador = 0
for i, P1 in enumerate(pauli_matrices):
    for j, P2 in enumerate(pauli_matrices):
        O = sp.kronecker_product(P1, P2)
        Eq = EDP(H, O, Ls, gammas)
        #print(f'{pauli_labels[i]}{pauli_labels[j]} {Eq}')
        saida =''
        if len(Eq) !=0 :
            saida  = f'LOSS_edo += (dX_dt[:,{countador}:{countador+1}] - (' 
            for k in range(len(Eq)):
                saida += f' +{sp.Mul(sp.symbols(Eq[k][1]),Eq[k][0], evaluate=False)}'
            saida += ')**2'
            print(saida)
            countador +=1

LOSS_edo += (dX_dt[:,0:1] - ( +IX*(-0.5*gamma3 - 2.0*gamma4) +(-1.0*JXZ)*XY +JXY*XZ +(-1.0*JYZ)*YY +(1.0*JYY)*YZ +(-1.0*JZZ)*ZY +(1.0*JZY)*ZZ)**2
LOSS_edo += (dX_dt[:,1:2] - ( +IY*(-0.5*gamma3 - 2.0*gamma4) +(1.0*JXZ)*XX +(-1.0*JXX)*XZ +JYZ*YX +(-1.0*JYX)*YZ +(1.0*JZZ)*ZX +(-JZX)*ZZ)**2
LOSS_edo += (dX_dt[:,2:3] - ( +II*(-1.0*gamma3) +IZ*(-1.0*gamma3) +(-1.0*JXY)*XX +(1.0*JXX)*XY +(-1.0*JYY)*YX +JYX*YY +(-1.0*JZY)*ZX +(1.0*JZX)*ZY)**2
LOSS_edo += (dX_dt[:,3:4] - ( +XI*(-0.5*gamma1 - 2.0*gamma2) +(-1.0*JZX)*YX +(-1.0*JZY)*YY +(-1.0*JZZ)*YZ +JYX*ZX +(1.0*JYY)*ZY +(1.0*JYZ)*ZZ)**2
LOSS_edo += (dX_dt[:,4:5] - ( +IY*(-1.0*JXZ) +IZ*(1.0*JXY) +XX*(-0.5*gamma1 - 2.0*gamma2 - 0.5*gamma3 - 2.0*gamma4) +(-1.0*JZX)*YI +(1.0*JYX)*ZI)**2
LOSS_edo += (dX_dt[:,5:6] - ( +IX*JXZ +IZ*(-1.0*JXX) +XY*(-0.5*gamma1 - 2.0*gamma2 - 0.5*gamma3 - 2.0*gamma4) +(-1.0*JZY)*YI +(1.0*JYY)*ZI)**2
LOSS_edo += (dX_dt[:,6:7] - ( +IX*(-1.0*JXY) +IY*(1.0*JXX) +XI*(-1.0*gamma3) +XZ*(-0.5*gamma1 - 2.0*gamma2 - 1.0*gamma3) +(

In [28]:
countador

15